In [1]:
# Import Dependencies

import json
import requests
import pandas as pd
import numpy as np
from config import socrata, socratasecret

In [2]:
# Storing base url for CDC API

base_url = 'https://data.cdc.gov/resource/n8mc-b4w4.json'

In [3]:
# Build Query URL? Using socrata and socratasecret?

# query_url = base_url + socrata
# ^^ skeleton frame for it



In [4]:
# Get Data

cdc_response = requests.get(base_url)
cdc_json = cdc_response.json()

print(f"Here's what CDC sent: {cdc_json}.")

Here's what CDC sent: [{'case_month': '2022-05', 'res_state': 'NY', 'state_fips_code': '36', 'res_county': 'NEW YORK', 'county_fips_code': '36061', 'age_group': '18 to 49 years', 'sex': 'Female', 'race': 'NA', 'ethnicity': 'NA', 'case_positive_specimen': '0', 'process': 'Missing', 'exposure_yn': 'Missing', 'current_status': 'Laboratory-confirmed case', 'symptom_status': 'Unknown', 'hosp_yn': 'Missing', 'icu_yn': 'Missing', 'death_yn': 'Missing'}, {'case_month': '2020-12', 'res_state': 'NY', 'state_fips_code': '36', 'res_county': 'NASSAU', 'county_fips_code': '36059', 'age_group': '18 to 49 years', 'sex': 'Female', 'race': 'NA', 'ethnicity': 'NA', 'case_positive_specimen': '3', 'case_onset_interval': '0', 'process': 'Missing', 'exposure_yn': 'Missing', 'current_status': 'Laboratory-confirmed case', 'symptom_status': 'Symptomatic', 'hosp_yn': 'Missing', 'icu_yn': 'Missing', 'death_yn': 'No'}, {'case_month': '2020-03', 'res_state': 'NY', 'state_fips_code': '36', 'res_county': 'BRONX', 'co

In [5]:
# explore data --->

cdc_df = pd.DataFrame(cdc_json)
cdc_df.head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_onset_interval,underlying_conditions_yn
0,2022-05,NY,36,NEW YORK,36061,18 to 49 years,Female,NA,NA,0,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN
1,2020-12,NY,36,NASSAU,36059,18 to 49 years,Female,NA,NA,3,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,No,0,NaN
2,2020-03,NY,36,BRONX,36005,65+ years,Female,Black,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Unknown,Yes,Unknown,Unknown,NaN,Yes
3,2022-07,CA,06,SAN DIEGO,06073,18 to 49 years,Male,Unknown,Unknown,NaN,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN
4,2020-11,OH,39,NA,NA,50 to 64 years,Male,White,Non-Hispanic/Latino,NaN,Clinical evaluation,Unknown,Probable Case,Symptomatic,Missing,Missing,Unknown,0,Yes


In [17]:
cdc_df.describe()

# note the top values of 'missing' for columns:
# process, exposure_yn, symptom_status (unk), hosp_yn, icu_yn, death_yn  
# with high frequencies too
# Per Mandy's suggestion would make most sense to drop columns

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_onset_interval,underlying_conditions_yn
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,371,1000,1000,1000,1000,1000,1000,1000,353,97
unique,18,10,10,19,19,4,2,6,5,10,6,3,2,4,4,4,5,4,1
top,2022-01,NY,36,SAN DIEGO,06073,18 to 49 years,Female,White,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,0,Yes
freq,209,264,264,54,54,532,630,315,358,335,922,960,934,390,458,856,657,350,97


In [21]:
cdc_df.drop(['process', 'exposure_yn', 'symptom_status', 'hosp_yn', 'icu_yn', 'death_yn'], axis=1, inplace=True)
cdc_df.head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen,current_status,case_onset_interval,underlying_conditions_yn
0,2022-05,NY,36,NEW YORK,36061,18 to 49 years,Female,NA,NA,0,Laboratory-confirmed case,NaN,NaN
1,2020-12,NY,36,NASSAU,36059,18 to 49 years,Female,NA,NA,3,Laboratory-confirmed case,0,NaN
2,2020-03,NY,36,BRONX,36005,65+ years,Female,Black,Non-Hispanic/Latino,0,Laboratory-confirmed case,NaN,Yes
3,2022-07,CA,06,SAN DIEGO,06073,18 to 49 years,Male,Unknown,Unknown,NaN,Laboratory-confirmed case,NaN,NaN
4,2020-11,OH,39,NA,NA,50 to 64 years,Male,White,Non-Hispanic/Latino,NaN,Probable Case,0,Yes


In [33]:
cdc_df['case_positive_specimen'].isnull().sum()

629

In [34]:
cdc_df['case_onset_interval'].isnull().sum()

647

In [35]:
cdc_df['underlying_conditions_yn'].isnull().sum()

903

In [36]:
cdc_df['age_group'].value_counts()

18 to 49 years    532
0 - 17 years      166
65+ years         157
50 to 64 years    145
Name: age_group, dtype: int64

In [37]:
cdc_df['sex'].value_counts()

Female    630
Male      370
Name: sex, dtype: int64

In [38]:
cdc_df['race'].value_counts()
# Note 'NA' and 'Missing' *AND* 'Unknown'

White      315
Unknown    194
NA         184
Black      174
Missing    106
Asian       27
Name: race, dtype: int64

In [45]:
cdc_df['ethnicity'].value_counts()

Non-Hispanic/Latino    358
Unknown                301
NA                     184
Hispanic/Latino        105
Missing                 52
Name: ethnicity, dtype: int64

In [46]:
cdc_df['case_month'].value_counts()

2022-01    209
2021-12    106
2022-05     70
2020-06     56
2020-03     53
2020-11     53
2021-03     53
2021-01     52
2020-07     52
2021-08     52
2020-08     49
2021-07     43
2020-05     35
2021-10     27
2022-07     27
2022-02     27
2022-08     26
2020-12     10
Name: case_month, dtype: int64

In [47]:
cdc_sample = cdc_df.dropna(how='any')

In [48]:
cdc_sample.describe()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen,current_status,case_onset_interval,underlying_conditions_yn
count,24,24,24,24,24,24,24,24,24,24,24,24,24
unique,5,3,3,4,4,3,2,3,3,5,2,1,1
top,2020-08,NV,32,CLARK,32003,50 to 64 years,Female,White,Hispanic/Latino,0,Laboratory-confirmed case,0,Yes
freq,17,20,20,20,20,18,23,21,20,13,23,24,24


In [49]:
cdc_sample.to_csv('resources/cdc_sample.csv')